[DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Disease Tracking Dashboard

This is a template for your DIY Disease Tracking Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from anytree import Node, RenderTree
from anytree.importer import JsonImporter, DictImporter
from IPython.display import clear_output
import ipywidgets as wdg
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import pickle
import requests
import time





In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100
global regions 
regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("mean_occBed_region.json", "rt") as INFILE:
    mean_occBed_region=json.load(INFILE)

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")



    data={}
    for dataset in rawdata:
        for entry in dataset:
            date=entry['date']
            value=entry['metric_value']
            geography=entry['geography']
            if date not in data:
                data[date]={}
            data[date][geography]=value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    print (startdate, ' to ', enddate)

    index=pd.date_range(startdate, enddate, freq='D')
    mean_occBed_region_df=pd.DataFrame(index=index, columns=regions)


    for date, values in data.items():
        for region, value in values.items():
            mean_occBed_region_df.at[parse_date(date), region] = value
    # fill in any remaining "holes" due to missing dates
    mean_occBed_region_df.fillna(0.0, inplace=True)

    return mean_occBed_region_df

    

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df1=wrangle_data(mean_occBed_region) # df is the dataframe for plotting

2022-01-01 00:00:00  to  2022-12-31 00:00:00


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the UKHSA API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    
    class APIwrapper:
        # class variables shared among all instances
        _access_point="https://api.ukhsa-dashboard.data.gov.uk/v2"         # v2??????????????????????????????????????????
        _last_access=0.0 # time of last api access
        root = Node("root")
        
        def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
            """ Init the APIwrapper object, constructing the endpoint from the structure
            parameters """
            # build the path with all the required structure parameters. You do not need to edit this line,
            # parameters will be replaced by the actual values when you instantiate an object of the class!
            url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                    f"{geography_type}/geographies/{geography}/metrics/{metric}")
            # our starting API endpoint
            self._start_url=APIwrapper._access_point+url_path
            self._filters=None
            self._page_size=-1
            # will contain the number of items
            self.count=None



        def get_page(self, filters={}, page_size=5):
            """ Access the API and download the next page of data. Sets the count
            attribute to the total number of items available for this query. Changing
            filters or page_size will cause get_page to restart from page 1. Rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365); use the default value 
            for debugging your structure and filters. """
            # Check page size is within range
            if page_size>365:
                raise ValueError("Max supported page size is 365")
            # restart from first page if page or filters have changed
            if filters!=self._filters or page_size!=self._page_size:
                self._filters=filters
                self._page_size=page_size
                self._next_url=self._start_url
            # signal the end of data condition
            if self._next_url==None: 
                return [] # we already fetched the last page
            # simple rate limiting to avoid bans
            curr_time=time.time() # Unix time: number of seconds since the Epoch
            deltat=curr_time-APIwrapper._last_access
            if deltat<0.33: # max 3 requests/second
                time.sleep(0.33-deltat)
            APIwrapper._last_access=curr_time
            # build parameter dictionary by removing all the None
            # values from filters and adding page_size
            parameters={x: y for x, y in filters.items() if y!=None}
            parameters['page_size']=page_size
            # the page parameter is already included in _next_url.
            # This is the API access. Response is a dictionary with various keys.
            # the .json() method decodes the response into Python object (dictionaries,
            # lists; 'null' values are translated as None).
            response = requests.get(self._next_url, params=parameters).json()
            # update url so we'll fetch the next page
            self._next_url=response['next']
            self.count=response['count']
            # data are in the nested 'results' list
            return response['results'] 

        def get_all_pages(self, filters={}, page_size=365):
            """ Access the API and download all available data pages of data. Sets the count
            attribute to the total number of items available for this query. API access rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365), and controls the trade-off
            between time to load a page and number of pages; the default should work well 
            in most cases. The number of items returned should in any case be equal to 
            the count attribute. """
            data=[] # build up all data here
            while True:
                # use get_page to do the job, including the pacing
                next_page=self.get_page(filters, page_size)
                if next_page==[]:
                    break # we are done
                data.extend(next_page)
            return data


        # /themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/{geography_type}/geographies/{geography}/metrics/{metric}
        #REDO so starts from _access_point="https://api.ukhsa-dashboard.data.gov.uk" 
        def add_branch_to_node(self, upper_node = root, link = _access_point+'/themes/'):
            importer = DictImporter()
            curr_time=time.time() # Unix time: number of seconds since the Epoch
            deltat=curr_time-APIwrapper._last_access
            if deltat<0.33: # max 3 requests/second
                time.sleep(0.33-deltat)
            APIwrapper._last_access=curr_time

            branches = requests.get(link).json()
            
            # 
            if branches:        # make this better api stuff, 200?
                for branch in branches:
                    new_node = importer.import_(branch)
                    new_node.parent = upper_node
                    new_node.name = branch['name']
                    #print(RenderTree(root))
                    curr_time=time.time() # Unix time: number of seconds since the Epoch
                    deltat=curr_time-APIwrapper._last_access
                    if deltat<0.33: # max 3 requests/second
                        time.sleep(0.33-deltat)
                    APIwrapper._last_access=curr_time  
                    
                    next_level_link = branch['link']  # get the link to the next section in the api
                    options_link = requests.get(next_level_link).json()
                    if list(options_link[0].keys())[0] == 'metrics':        # bottom of tree
                        continue                                            # move on to next branch
                    options_link = list(options_link[0].values())[0]
                    
                    self.add_branch_to_node(new_node, options_link)

            return
    
    # This Builds The Tree !!It Takes Over 30 min To Run on Local Machine!!
    # api.add_branch_to_node()
    
    # Load the tree from a pickle file instead
    global root
    with open('full_tree.pkl', 'rb') as f:
        root = pickle.load(f)

    filters={"stratum" : None, # Smallest subgroup a metric can be broken down into e.g. ethnicity, testing pillar
        "age": None, # Smallest subgroup a metric can be broken down into e.g. 15_44 for the age group of 15-44 years
        "sex": None, #  Patient gender e.g. 'm' for Male, 'f' for Female or 'all' for all genders
        "year": 2022, #  Epi year of the metrics value (important for annual metrics) e.g. 2020
        "month": None, # Epi month of the metric value (important for monthly metrics) e.g. 12
        "epiweek" :None, # Epi week of the metric value (important for weekly metrics) e.g. 30
        "date" : None, # The date which this metric value was recorded in the format YYYY-MM-DD e.g. 2020-07-20
        "in_reporting_delay_period": None # Boolean indicating whether the data point is considered to be subject to retrospective updates
    }
    
    structure={"theme": "infectious_disease", 
        "sub_theme": "respiratory",
        "topic": "COVID-19", "metric": "COVID-19_healthcare_occupiedBedsRollingMean", "geography_type": "NHS Region"} #, "geography": "England"}
    
    
    mean_occBed_region = []
    for idx, r in enumerate(regions):
        structure["geography"] = r
        api=APIwrapper(**structure)
        mean_occBed_region.append(api.get_all_pages(filters))                    # FILTERED!!!! 2022 only
        print(f"Data points expected: {api.count}")
        print(f"Data points retrieved: {len(mean_occBed_region[idx])}")
        
        
    #timeseriesdf = wrangle_data(mean_occBed_region)   


    
    #return timeseriesdf # return data read from the API
    return mean_occBed_region

In [ ]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df1
    global regions 
    regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # The function needs to be adapted to your graph; you can omit this call
    # in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. If you are 
    # implementing error handling, you can use icons "unlink" or "times" and 
    # change the button text to "Unavailable" when the api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Engage', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='danger', description='Engage', icon='exclamation-triangle', style=ButtonStyle(), tooltip=…

FileNotFoundError: [Errno 2] No such file or directory: 'full_tree.pkl'

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [13]:
def update_plot(selected_regions):
    #selected_regions = [region for region, checkbox in region_checkboxes.items() if checkbox.value]
    selected_regions = list(selected_regions)
    df1[selected_regions].plot()
    # ax = df1[selected_regions].plot()
    # ax.set_title("test title")
    # ax.xaxis.set_minor_locator(mdates.MonthLocator())
    # ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    # ax.tick_params(axis='x', which='minor', rotation=45)
    plt.show()



reg = wdg.SelectMultiple(
    options=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    value=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    #rows=10,
    description='Regions',
    disabled=False
)
reg.layout.height = '120px'

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=reg.value
    if current==reg.options[0]:
        other=reg.options[1]
    else:
        other=reg.options[0]
    reg.value=other # forces the redraw
    reg.value=current # now we can change it back
    
controls=wdg.HBox([apibutton, reg]) 

# connect the plotting function and the widget    
graph=wdg.interactive_output(update_plot, {'selected_regions': reg})

# actually displays the graph
display(controls, graph)




# def refresh_graph():
#     """ We change the value of the widget in order to force a redraw of the graph;
#     this is useful when the data have been updated. This is a bit of a gimmick; it
#     needs to be customised for one of your widgets. """
    
#     # current=region_checkboxes[list(region_checkboxes.keys())[0]].value
#     # if current==region_checkboxes[list(region_checkboxes.keys())[0]].options[0]:
#     #     other=region_checkboxes[list(region_checkboxes.keys())[0]].options[1]
#     # else:
#     #     other=region_checkboxes[list(region_checkboxes.keys())[0]].options[0]
#     # region_checkboxes[list(region_checkboxes.keys())[0]].value=other # forces the redraw
#     # region_checkboxes[list(region_checkboxes.keys())[0]].value=current # now we can change it back
#       change to not checkbox?
#     current=region_checkboxes[list(region_checkboxes.keys())[0]].value
#     region_checkboxes[list(region_checkboxes.keys())[0]].value = not current
#     region_checkboxes[list(region_checkboxes.keys())[0]].value = current
    

# region_checkboxes = {region: wdg.Checkbox(value=True, description=region) for region in regions}
# for checkbox in region_checkboxes.values():
#     checkbox.observe(update_plot, 'value')

# # Create two horizontal rows of checkboxes
# row1 = wdg.HBox(list(region_checkboxes.values())[:4])
# row2 = wdg.HBox(list(region_checkboxes.values())[4:])

# # try replacing HBox with a VBox!!
# controls=wdg.HBox([apibutton, wdg.VBox([row1, row2])])

# # capture output in widget graph   
# graph=wdg.interactive_output(update_plot, {'change': region_checkboxes[regions[0]]})

# display(controls, graph)

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."